<a href="https://colab.research.google.com/github/harisuguru06/Automatic-Image-Captioning/blob/master/Model_Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Deployment

Install Flask and Ngrok

In [22]:
!pip install flask-ngrok
!pip install flask==0.12.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Import keras libs

In [23]:
!pip install tensorflow
!pip install keras
!pip install pillow
!pip install numpy
!pip install tqdm
!pip install keras.utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Load google drive and create virtual folder for website


In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import ML libraries

In [25]:
import numpy as np
from pickle import dump, load
from PIL import Image
from keras.models import Model, load_model
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [26]:
def extract_features(filename, model):
        try:
            image = Image.open(filename)
        except:
            print("ERROR: Couldn't open image! Make sure the image path and extension is correct")
        image = image.resize((299,299))
        image = np.array(image)
        # for images that has 4 channels, we convert them into 3 channels
        if image.shape[2] == 4: 
            image = image[..., :3]
        image = np.expand_dims(image, axis=0)
        image = image/127.5
        image = image - 1.0
        feature = model.predict(image)
        return feature

In [27]:
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
         return word
    return None

def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'start'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        pred = model.predict([photo,sequence], verbose=0)
        pred = np.argmax(pred)
        word = word_for_id(pred, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'end':
            break
    return in_text

1.Initalize and run Flask and Ngork

2.Load model and predict it

3.Updated result back to use

In [28]:
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok

/drive/ngrok-ssh
--2022-08-14 03:04:55--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.161.241.46, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  56.7MB/s    in 0.2s    

2022-08-14 03:04:56 (56.7 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip


In [29]:
!/ngrok authtoken 2DI4bkOAQIhGOsc75CHmWQFEVuq_5pFDKAMNaBtb6eH2dLTNW

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [30]:
cd '/content/drive/MyDrive/website'

/content/drive/MyDrive/website


In [38]:
# flask_ngrok_example.py
import os
from flask import Flask, request, render_template, url_for
from flask_ngrok import run_with_ngrok

UPLOAD_FOLDER = '/content/drive/MyDrive/website/uploads'
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg'])


app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/")
def home():
    return render_template('index.html')

@app.route('/getprediction', methods=['POST'])
def getprediction():
    #model prediction goes here

    description = ''
    if request.method == 'POST':
        if 'file1' not in request.files:
            return 'there is no file1 in form!'
        file1 = request.files['file1']
        path = os.path.join(app.config['UPLOAD_FOLDER'], file1.filename)
        file1.save(path)

        # Predict y-hat
        img_path = path
        max_length = 32
         
        
        tokenizer = load(open('/content/drive/MyDrive/website/model/tokenizer.p',"rb"))
        model = load_model('/content/drive/MyDrive/website/model/model_9.h5')
        xception_model = Xception(include_top=False, pooling="avg")
        photo = extract_features(img_path, xception_model)
        img = Image.open(img_path)
        description = generate_desc(model, tokenizer, photo, max_length)
        description = description.replace("start ", "")
        description = description.replace(" end", "")

        #return path

        #return 'ok'
        return render_template('index.html', output='caption: {}'.format(description), picture=path)

if __name__ == '__main__':
    app.run()  # If address is in use, may need to terminate other sessions:
               # Runtime > Manage Sessions > Terminate Other Sessions

INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://a0a7-34-148-31-45.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [14/Aug/2022 03:35:09] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Aug/2022 03:35:09] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [14/Aug/2022 03:35:36] "POST /getprediction HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Aug/2022 03:35:36] "GET /content/drive/MyDrive/website/uploads/COCO_val2014_000000324670.jpg HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [14/Aug/2022 03:36:04] "GET /uploads/COCO_val2014_000000324670.jpg HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [14/Aug/2022 03:36:14] "GET /uploads/COCO_val2014_000000324670.jpg HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [14/Aug/2022 03:36:25] "GET /uploads/COCO_val2014_000000324670.jpg HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [14/Aug/2022 03:36:33] "GET /uploads/COCO_val2014_000000324670.jpg HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [14/Aug/2022 03:36:43] "GET /uploads/COCO_val2014_000000324670.jpg HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [14/Aug/2022 03:36:51] "GET /